In [15]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [16]:
import pandas as pd

In [17]:
%%R
data_folder = './'
setwd(data_folder)

In [9]:
%%R
library(proteus)

proteinGroupsFile <- "./proteinGroups.txt"
meta <- read.csv("./metadata.csv")
prot.MQ <- readProteinGroups(proteinGroupsFile, meta)

#It is possible to read these data directly into Proteus and skip peptide and protein aggregation steps.
prodat.med <- normalizeData(prot.MQ)

res <- limmaDE(prodat.med, sig.level=0.05)
r <- res[which(res$significant), c("protein", "logFC", "adj.P.Val")]
write.csv(res, "proteus_median.csv")

In [18]:
proteus_median = pd.read_csv("./proteus_median.csv", header=0, sep=",", index_col=0)
proteus_median.dropna(subset=["logFC", "P.Value"], how="any", inplace=True)
positive =  proteus_median[(abs(proteus_median["logFC"]) >1) &(proteus_median["adj.P.Val"] <0.05) ]
negative = proteus_median[(abs(proteus_median["logFC"]) <=1) | (proteus_median["adj.P.Val"] >=0.05) ]

no_difference_ups = ["P00167", "P01133", "P02144", "P04040", "P15559", "P62937", "P63165",  "Q06830"]
difference_ups =  ["P01579", "O76070", "P63279", "P68871","P01127", "P01008", "P62988", "P10599", "P02787", "P99999", "P12081", "P51965","P10636-8", 
       "P01031", "P09211", "P02788", "P41159", "O00762", "P05413", "P00441", "P00918", "P08758", "P00915", "P01344", "P69905", 
        "P00709", "P55957", "P08263", "P61769", "P10145", "P16083", "P61626", "P02741", "P06732",
        "P01375", "P06396", "P02753", "P01112", "Q15843", "P02768"]


TP = len(positive[positive["protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

33
6
2191
1


In [10]:
%%R

#It is possible to read these data directly into Proteus and skip peptide and protein aggregation steps.
prodat.med <- normalizeData(prot.MQ, norm.fun=limma::normalizeQuantiles)

res <- limmaDE(prodat.med, sig.level=0.05)
r <- res[which(res$significant), c("protein", "logFC", "adj.P.Val")]
write.csv(res, "proteus_normalizeQuantiles.csv")

In [19]:
proteus_normalizeQuantiles = pd.read_csv("./proteus_normalizeQuantiles.csv", header=0, sep=",", index_col=0)
proteus_normalizeQuantiles.dropna(subset=["logFC", "P.Value"], how="any", inplace=True)
positive =  proteus_normalizeQuantiles[(abs(proteus_normalizeQuantiles["logFC"]) >1) &(proteus_normalizeQuantiles["adj.P.Val"] <0.05) ]
negative = proteus_normalizeQuantiles[(abs(proteus_normalizeQuantiles["logFC"]) <=1) | (proteus_normalizeQuantiles["adj.P.Val"] >=0.05) ]
TP = len(positive[positive["protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

33
6
2191
1
